<a href="https://colab.research.google.com/github/Janek9527/KI_Lab_Project/blob/master/LAB_Project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://kilab.blob.core.windows.net/code/fishygame.zip
!unzip fishygame.zip

--2021-06-04 08:14:29--  https://kilab.blob.core.windows.net/code/fishygame.zip
Resolving kilab.blob.core.windows.net (kilab.blob.core.windows.net)... 20.38.118.132
Connecting to kilab.blob.core.windows.net (kilab.blob.core.windows.net)|20.38.118.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495570 (484K) [application/zip]
Saving to: ‘fishygame.zip.1’

fishygame.zip.1     100%[===================>] 483.96K   407KB/s    in 1.2s    

2021-06-04 08:14:31 (407 KB/s) - ‘fishygame.zip.1’ saved [495570/495570]

Archive:  fishygame.zip
replace fishygame/game_class.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: fishygame/game_class.py  
  inflating: fishygame/.gitignore    
  inflating: fishygame/resources/fish_textures - new/idle/1.png  
  inflating: fishygame/resources/restart_button/mouse_in/1.mdp  
  inflating: fishygame/resources/restart_button/mouse_in/1.png  
  inflating: fishygame/resources/restart_button/mouse_pressed/1.mdp  
 extracting: fishy

In [2]:
!pip3 install arcade

In [3]:
%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(960, 540))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '960x540x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '960x540x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [39]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x = self.flatten(x)
        h_relu = F.relu(self.linear1(x))
        return self.linear2(h_relu)

In [52]:
state_dim = 15 * 5
hidden_dim = 200
action_count = 4

model = Net(state_dim, hidden_dim, action_count)
activation = nn.Softmax(dim=1)

In [8]:
import sys
sys.path.append("fishygame")

In [9]:
from fishygame import game_class

In [10]:
initialState = [[0.0e+00, 0.0e+00, 4.8e+02, 2.7e+02, 3.0e-01],        
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00]]

In [57]:
from game_class import Game

def get_episodes(num_episodes, max_episode_length, model, old_reward):
    episodes = []
    while len(episodes) < num_episodes:
        observation = initialState
        game = Game()
        states = []
        sum_reward = 0
        rewards = []
        actions = []
        for i in range(max_episode_length):
            # Select random action
            activation_proba = activation(model(torch.tensor([observation]).float()))
            action_probs = activation_proba.data.numpy()[0]
            action = np.random.choice(len(action_probs), p=action_probs)
            
            observation, reward, done = game.on_update(action)
            
            sum_reward += reward
            states.append(observation)
            rewards.append(reward)
            actions.append(action)

            if(done):
                break

        if sum_reward > old_reward:
          episodes.append((sum_reward, states, rewards, actions))

    return episodes


In [58]:
def filter_episodes(episodes, percentile):

  episodes.sort(key=lambda x:x[0])

  return episodes[:int(len(episodes) * percentile)]

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


mean_reward = 0
epoch = 0
old_reward = 0
while mean_reward < 100:
  #generate 100 episodes
  print('Epoch %s' % epoch)
  epoch += 1
  episodes = filter_episodes(get_episodes(10, 5000, model, old_reward), 0.5)

  print(f"Found {len(episodes)} episodes with reward > 0")
  
  sum_old_reward = 0
  
  for episode in episodes:
    sum_old_reward += episode[0]
    target_action_batch = torch.tensor(episode[3])
    print(torch.LongTensor(target_action_batch))
    predicted_action_batch = model(torch.tensor(episode[1]).float())
    print(torch.LongTensor(predicted_action_batch.argmax(dim=1)))
    # print(action)
    # print(target_action)
    # print(predicted_actions)

  #print(torch.stack(predicted_action_batch))
  #print(torch.stack(target_action_batch))
    loss = loss_function(predicted_action_batch, target_action_batch)
    print(f"Loss: {loss}")
    print(f"Reward: {episode[0]}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  old_reward = sum_old_reward / len(episodes)

Epoch 0
Found 5 episodes with reward > 0
tensor([3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3,
        3, 1, 3, 3, 1, 3, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
!rm -r fishygame
!rm fishygame.zip